# Algoritmo QR
Es una técnica de reducción de matriz utilizada para determinar los valores propios o eigenvalores de una matriz simétrica.
Para aplicar el algoritmo es necesario tener una matriz simétrica diagonal. Es decir, una matriz con elementos con valor distinto de 0 ubicados unicamente en la diagonal principal o en las subdiagonales adyacentes.

Si la matriz de la que se dispone no se trata de una matriz simétrica diagonal, debe aplicarse el método de Householder para encontrar una matriz con las características deseadas.

Sea 'a' un arreglo que contiene los valores de la diagonal central.
Sea 'b' un arreglo que contiene los valores de las subdiagonales.

\begin{equation}
a =
\begin{bmatrix}
a_1 & a_2 & a_3 & ... & a_n
\end{bmatrix}
\end{equation}

\begin{equation}
b =
\begin{bmatrix}
b_2 & b_3 & ... & b_n
\end{bmatrix}
\end{equation}

Se puede denotar una matriz simétrica tridiagonal de la siguiente manera:

\begin{equation}
\begin{bmatrix}
a_1 & b_2 & 0 & 0\\
b_2 & a_2 & b_3 & 0\\
0 & b_3 & a_3 & b_n\\
0 & 0 & b_n & a_n
\end{bmatrix}
\end{equation}

Si un elemento $b_n$ tiene un valor de 0, entonces la matriz 1x1 $[a_n]$ inmediatamente produce un eigenvalor $a_n$.

Cuando un elemento $b_j$ = 0 para algunas j, donde: \begin{equation} 2 < j < n \end{equation}

El algoritmo reduce el problema particionando la matriz A en dos matrices:

\begin{equation}
\begin{bmatrix}
a_1 & b_2 & 0 & 0\\
b_2 & a_2 & b_3 & 0\\
0 & b_3 & a_3 & b_{j-1}\\
0 & 0 & b_{j-1} & a_{j-1}
\end{bmatrix}
\end{equation}
\begin{equation}
\begin{bmatrix}
a_j & b_{j+1} & 0 & 0\\
b_{j+1} & a_{j+1} & b_{j+2} & 0\\
0 & b_{j+2} & a_{j+2} & b_n\\
0 & 0 & b_n & a_n
\end{bmatrix}
\end{equation}




Luego se forma una sucesión de matrices: 

\begin{equation}
A= A^1, A^2, A^3, ...
\end{equation}

Donde un elemento $A^i$ se factoriza como un producto $\begin{equation} A^i = Q^i R^i \end{equation}$, donde $Q^i$ es una matriz ortogonal y $R^i$ es una matriz triangular superior.
Asi mismo, un elemento $A^{i+1}$ se define como el producto de $R^i$ y $Q^i$ en dirección inversa $\begin{equation} A^i = R^i Q^i\end{equation}$. Debido a que el elemento $Q^i$ es ortogonal, es posible expresar $R^i$ como $\begin{equation} R^i = Q^{i^t} A^i \end{equation}$, por lo tanto:
\begin{equation} 
A^{i+1} = R^i Q^i = (Q^{i^t} A^i) Q^i = Q^{i^t} A^i Q^i 
\end{equation}

De esta manera, se garantiza que $A^{i+1}$ es simétrica con los mismos valores propios que $A^i$ y que se trata de una matriz tridiagonal.

La factorización de $A^1 = Q^1 R^1$ usa un producto de n - 1 matrices de rotación para construir la matriz $R^1 = P_n P_{n-1} ... P_2 A^1$.

La matriz $P_k$ se selecciona de manera que $A_k^1 = P_k A_{k-1}^1$ en la posición $[k, k - 1]$ sea igual a cero.
